In [1]:
import numpy as np
from copy import copy

In [2]:
def taxi_price(distance, time_interval):
    real_distance = distance
    final_price = 0
    min_price = 0.8 * time_interval
    
    if real_distance < 2:
        final_price = 10 + min_price
    elif real_distance > 25:
        final_price = 10 + (25 - 2) * 2.6 + (real_distance - 25) * 1.3 * 2.6 + min_price
    elif real_distance > 50:
        final_price = 10 + (25 - 2) * 2.6 + 25 * 1.3 * 2.6 + (real_distance - 50) * 1.6 * 2.6 + min_price
    
    return final_price

In [3]:
# generate taxi coming condition
def taxiAndCustomerFlow(mu_t, mu_c, n):
#     taxi flow = new_arrived taxi + taxi that return
    return np.random.poisson(mu_t, [1,n]), np.random.poisson(mu_c, [1,n])

In [4]:
def getTripDistribution():
#     10km 10min
    distance = np.random.exponential(10)
    return distance, distance * 2

In [5]:
# if taxi goes too long away, then drop this instance
def dropIfLong(distance, taxi, dropTaxiList, ceil):
    new_drop_list = []
    if distance > ceil:
        dropTaxiList.append(taxi)
        new_drop_list = copy(dropTaxiList)
        return True, new_drop_list
    else:
        new_drop_list = copy(dropTaxiList) 
        return False, new_drop_list

In [6]:
class Taxi:
    money = 0
    backtime = 0
    def __init__(self, index):
        self.index = index
    def earnMoney(self, new_money):
        self.money += new_money
    def returnMoney(self):
        return self.money
    def setBackTime(self, time):
        self.backtime = time
    def subBackTime(self, time):
        self.backtime -= time
    def getBackTime(self):
        return self.backtime
    def checkBack(self):
        return self.backtime <= 0

In [7]:
# use time_back to sort the taxi
import operator
def checkTaxiBack(out_list, currentTaxiNum):
#     print('here', out_list)
    length = len(out_list)
    
    if(length == 0):
        return ([],[],currentTaxiNum)
    cmpfun = operator.attrgetter('money')
    new_out_list = sorted(out_list,key=cmpfun)
    #check how many cars are back, store in back taxis
    back_taxis = []
    #if back number larger than currentTaxiNum, directly return 0
    cnt = 0
    for taxi in new_out_list:
        if taxi.checkBack():
            cnt += 1
#             add the taxi to back and remove from the out list
            back_taxis.append(taxi)
            new_out_list.remove(taxi)
    new_come_taxi = 0
    if cnt <= currentTaxiNum:
        new_come_taxi = currentTaxiNum - cnt
    else:
        return new_out_list, back_taxis, new_come_taxi
        
    return new_out_list, back_taxis, new_come_taxi

In [8]:
def allocateTaxi(queue_list, number):
#     start allocation
    allocate_taxis = []
    
    if(number > 0):
        for i in range(number):
            allocate_taxis.append(queue_list[i])
#     print('new_queue_list',number, queue_list[number:])
    temp_list = queue_list[number:]
    new_queue_list = copy(temp_list)
#     print('new_queue_list', new_queue_list, allocate_taxis)
    return new_queue_list, allocate_taxis

In [9]:
def refreshOutTaxis(out_list, time):
    new_list = copy(out_list)
    if len(new_list) == 0:
        return new_list
    else:
        for i in out_list:
            i.subBackTime(time)
        return new_list

In [10]:
def setTripToCar(p_alloc_taxi_list, drop_taxi_list, ceil):
    alloc_taxi_list = []
    new_drop_taxi_list = []
    for i in p_alloc_taxi_list:
        new_trip_distance, new_trip_time = getTripDistribution()
        trip_money = taxi_price(new_trip_distance, new_trip_time) 
        i.earnMoney(trip_money)
        i.setBackTime(new_trip_time)
#         print('set trip to taxi',new_trip_distance, i, drop_taxi_list)
        status, new_drop_taxi_list = dropIfLong(new_trip_distance, i, drop_taxi_list, ceil)
        # not dropped
        if(not status):
            alloc_taxi_list.append(i)
    return alloc_taxi_list, new_drop_taxi_list

In [11]:
# def refreshInQueueTaxi(taxi_queue):
#     if len(taxi_queue) <= 1:
#         return
#     else:
#         logit = 1 / (1 + np.exp(1))
#         for taxi in taxi_queue:
#             time_now = taxi.getBackTime()
#             if time_now < 0:
#                 taxi.earnMoney(-10 * np.exp(2 * np.log(abs(time_now))))
        

In [12]:
def refreshInQueueTaxi(taxi_queue, alpha, beta):
    if len(taxi_queue) <= 1:
        return
    else:
        logit = 1 / (1 + np.exp(1))
        for taxi in taxi_queue:
            time_now = taxi.getBackTime()
            if time_now < 0:
                taxi.earnMoney(-beta * np.exp(alpha * np.log(abs(time_now))))
        

In [13]:
def algorithm(p_taxi_queue_list, back_taxis):
    taxi_queue_list = copy(p_taxi_queue_list)
    back_taxis_list = copy(back_taxis)
#     print('algorithm', taxi_queue_list)
    # insert cars
    temp_list = taxi_queue_list + back_taxis_list
    cmpfun = operator.attrgetter('backtime')
    taxi_queue_list = sorted(temp_list,key=cmpfun)
    # back new taxi queue
    new_taxi_queue_list =  copy(taxi_queue_list)
    return new_taxi_queue_list

In [14]:
# we generate taxi and cust flow.
# Unit of T is 5 min, so we simulate a 500 min case
# taxi can pickup customer if they are in the front of the queue
# we deemed that there exist 5 port for the taxi to pick customer up
def simulation(alpha, beta):
    T = 1000 # simulation time
    n = 2 # a taxi for 2 cust
    p = 10 # 10 port
    unit_time = 5 # 5min a round
    ceil = 10 # 10 km judge to 
    attenuation_rate = 0.8 #
    
    drop_taxi_list = [] # drop the car if too long
    
    taxi_queue_list = [] # waiting list for taxis
    cust_queue = 0 # waiting number for cust
    out_list = [] # for the taxi that are out of the air plane but short trip, going to return
    taxi_flow, cust_flow = taxiAndCustomerFlow(10, 30, T)
    taxi_flow = taxi_flow.tolist()[0]
    cust_flow = cust_flow.tolist()[0]
    
    wait_time = 5 # if taxi cannot get the 
    
    
        
    index_taxi = 0 # taxi index, use for create taxi object
    for i in range(T):
#         print(i)
#         print(taxi_queue_list, cust_queue, out_list)
        # Total flow of cust and taxi
        refreshInQueueTaxi(taxi_queue_list, alpha, beta)
        new_taxi_flow = taxi_flow[i]
        new_cust = cust_flow[i]
        
#         print(new_taxi_flow, new_cust) # for debug

        # refresh the status in out car, substract the interval
        out_list = copy(refreshOutTaxis(out_list, unit_time))
#         print('test_out_list', out_list) # for debug

        # check Taxi backs, and get new_come_taxi and back_taxi
        new_out_list, back_taxis, new_come_taxi = checkTaxiBack(out_list, new_taxi_flow)
        
#         print("back taxi", new_out_list, back_taxis, new_come_taxi) # for debug

        # refresh outlist
        out_list = new_out_list
#         print("out_list", out_list)
        # back taxis should add into the queue
        # algorithms here
#         print("algorithm", taxi_queue_list, back_taxis)
        taxi_queue_list = algorithm(taxi_queue_list, back_taxis)
        
        # newly come taxis
        # we deemed that all newly come taxi should append at the end of the queue list
        if new_come_taxi > 0:
            # generate new coming taxi
            for i in range(index_taxi, (index_taxi + new_come_taxi)):  
                new_taxi = Taxi(i)
                taxi_queue_list.append(new_taxi)
#         print("new taxi added", taxi_queue_list)
        # Now allocate the customer to the taxis
        # current customers
        current_cust = cust_queue + new_cust
        available_taxis_num = len(taxi_queue_list)
        # ports are enough
        if current_cust < (p * n):
            # car enough
            if current_cust < (available_taxis_num * n):
                # all cust in queue are gone
                require_taxi_num = round(current_cust / n)
                cust_queue = 0
                # allocate require taxi number
                new_taxi_queue_list, allocate_taxis = allocateTaxi(taxi_queue_list, require_taxi_num)
            # car not enough
            else:
                # refresh cust_queue
                cust_queue = current_cust - available_taxis_num * n
                new_taxi_queue_list, allocate_taxis = allocateTaxi(taxi_queue_list, available_taxis_num)
        # port not enough
        else:
            # car not enough for port
            if available_taxis_num < p:
                # refresh cust_queue
                cust_queue = current_cust - available_taxis_num * n
                new_taxi_queue_list, allocate_taxis = allocateTaxi(taxi_queue_list, available_taxis_num)
            # car enough for ports
            else:
                # cust_queue
                cust_queue = current_cust - p * n
                # allocate 5 car
                new_taxi_queue_list, allocate_taxis = allocateTaxi(taxi_queue_list, 5)
           
        # give trips to car
        allocate_taxis, new_drop_taxi_list = setTripToCar(allocate_taxis, drop_taxi_list, ceil)
        drop_taxi_list = copy(new_drop_taxi_list)
#         print("after allocation", allocate_taxis, drop_taxi_list, allocate_taxis)
        # add the out-trip taxi to the new_out_list
        out_list = copy(new_out_list + allocate_taxis) # reset the out list
        taxi_queue_list = copy(new_taxi_queue_list)
#         print('end',drop_taxi_list ,taxi_queue_list)
#         print(current_cust)
        current_cust *= attenuation_rate
    return drop_taxi_list, taxi_queue_list
    
        
        
    

In [15]:
def stripListByBound(low, high, value):
    new_list = []
    for i in value:
        if (low <= i) and (i <= high):
            new_list.append(i)
    return new_list
        

In [16]:
def getCompared(alpha, beta):
    result = simulation(alpha, beta)
    drop_taxis = result[0]
#     queue_taxis = result[1]
#     print(result)
    drop_taxi_payoffs = []
#     queue_taxi_payoffs = []
    
    for i in drop_taxis:
        drop_taxi_payoffs.append(i.returnMoney())
#     for j in queue_taxis:
#         queue_taxi_payoffs.append(j.returnMoney())
#     print(queue_taxi_payoffs)
    drop_taxi_payoffs = stripListByBound(-50, 25, drop_taxi_payoffs)
    std_drop = np.std(drop_taxi_payoffs)
#     std_queue = np.std(queue_taxi_payoffs)
#     return std_drop,drop_taxi_payoffs
    return std_drop

In [17]:
def get_grad(alpha, beta, delta=0.00001):
    payoff00 = getCompared(alpha, beta)
    payoff01 = getCompared(alpha, beta+delta)
    payoff10 = getCompared(alpha+delta, beta)
    grad01 = (payoff01-payoff00)/delta
    grad10 = (payoff10-payoff00)/delta
    return grad10, grad01 ### the gradient where alpha+delta / beta+delta
def grad_dct(start_alpha, start_beta, lr=0.1, threshold=0.01):
    total = 0
    grad = (get_grad(start_alpha, start_beta))
    nxt_a = start_alpha
    nxt_b = start_beta
    while abs(grad[0]) > threshold or abs(grad[1]) > threshold:
        print('a: ', nxt_a, 'b: ', nxt_b)
        total += 1
        grad = (get_grad(nxt_a, nxt_b))
#         print(grad)
        nxt_a = nxt_a-grad[0]*lr
        nxt_b = nxt_b-grad[1]*lr
        if total > 100000:
            break
    return nxt_a, nxt_b

In [18]:
getCompared(10,30)

7.2396605686902165

In [19]:
getCompared(163606.7169144583, -98964.01310787872)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


7.024316850945625

In [20]:
grad_dct(1,1)

a:  1 b:  1
a:  2956.985461920756 b:  565.2421046744506


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


a:  693.2553139547458 b:  -129.672424788365
a:  -609.540984698185 b:  -363.42380537149427
a:  -3362.1183370398076 b:  -2820.748994576394
a:  -4576.599214884698 b:  -1386.8116374792617
a:  -992.4545456245041 b:  808.1005631987832
a:  1298.4947499917002 b:  -690.3667296205622
a:  -2127.335553732478 b:  -4393.571726542128
a:  -580.7301279749477 b:  -7300.195753660999
a:  589.4309671714038 b:  -7135.546374324224
a:  -353.16650777346456 b:  -9790.945823509486
a:  -2793.990779660563 b:  -10286.368397290486
a:  -8459.198942910387 b:  -13440.809163336186
a:  -4802.19969916618 b:  -13690.997667465117


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

mpl.rcParams["font.sans-serif"] = ["FangSong"]
mpl.rcParams["axes.unicode_minus"] = False

x = drop_taxi_payoffs

plt.boxplot(x)

plt.xticks([1], ["随机数生成器AlphaRM"])
plt.ylabel("随机数值")
plt.title("随机数生成器抗干扰能力的稳定性")

plt.grid(axis="y", ls=":", lw=1, color="gray", alpha=0.4)

plt.show()

In [ ]:
len(result[1])